In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2,f_classif

Load the datasets

In [27]:
loan_applications = pd.read_csv('loan_applications.csv')
credit_features = pd.read_csv('credit_features_subset.csv')
data_dictionary = pd.read_csv('loan_data_dictionary.csv')

Explore the data

In [ ]:
print(loan_applications.info())
print(credit_features.info())
print(data_dictionary.head())

Merge datasets

In [ ]:
merged_data = pd.merge(loan_applications, credit_features, on='UID', how='inner')

Preprocessing

In [ ]:
merged_data.describe().T

In [ ]:
# Replace -1 with NaN for relevant columns
columns_to_replace = [
    'ALL_AgeOfOldestAccount', 'ALL_AgeOfYoungestAccount', 'ALL_CountActive',
    'ALL_CountClosedLast12Months', 'ALL_CountDefaultAccounts',
    'ALL_CountOpenedLast12Months', 'ALL_CountSettled', 'ALL_MeanAccountAge',
    'ALL_SumCurrentOutstandingBal', 'ALL_SumCurrentOutstandingBalExcMtg',
    'ALL_WorstPaymentStatusActiveAccounts'
]

for col in columns_to_replace:
    merged_data[col] = merged_data[col].replace(-1, np.nan)

print("Negative values replaced with NaN")

In [ ]:
# Function to identify outliers using IQR method
def identify_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Check for outliers in numeric columns
numeric_columns = merged_data.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    outliers = identify_outliers(merged_data, col)
    if not outliers.empty:
        print(f"Outliers in {col}: {len(outliers)} ({len(outliers)/len(merged_data)*100:.2f}%)")

In [ ]:
def cap_outliers(df, column, lower_percentile=0.01, upper_percentile=0.99):
    lower = df[column].quantile(lower_percentile)
    upper = df[column].quantile(upper_percentile)
    df[column] = df[column].clip(lower, upper)
    return df

# Cap outliers for most variables
columns_to_cap = [
    'ALL_AgeOfOldestAccount', 'ALL_AgeOfYoungestAccount', 'ALL_Count',
    'ALL_CountActive', 'ALL_CountClosedLast12Months', 'ALL_CountDefaultAccounts',
    'ALL_CountOpenedLast12Months', 'ALL_CountSettled', 'ALL_MeanAccountAge'
]

for col in columns_to_cap:
    merged_data = cap_outliers(merged_data, col)

# Log transform highly skewed variables
columns_to_log = ['ALL_SumCurrentOutstandingBal', 'ALL_SumCurrentOutstandingBalExcMtg']

for col in columns_to_log:
    merged_data[f'{col}_Log'] = np.log1p(merged_data[col])
    
print("Outliers handled through capping and log transformation")

# Check the results
for col in columns_to_cap + columns_to_log:
    print(f"\nSummary for {col}:")
    print(merged_data[col].describe())

In [ ]:
# Check for missing values
missing_values = merged_data.isnull().sum()
print("\nMissing values:")
print(missing_values[missing_values > 0])

# For this step, we'll just impute missing values with median
for col in merged_data.columns:
    if merged_data[col].isnull().sum() > 0:
        if merged_data[col].dtype != 'object':  # Only for numeric columns
            merged_data[col].fillna(merged_data[col].median(), inplace=True)

print("\nMissing values imputed with median")

In [ ]:
# For most fields, we'll use median imputation
columns_for_median_imputation = [
    'ALL_AgeOfOldestAccount', 'ALL_AgeOfYoungestAccount', 
    'ALL_CountClosedLast12Months', 'ALL_CountOpenedLast12Months',
    'ALL_CountSettled', 'ALL_MeanAccountAge',
    'ALL_SumCurrentOutstandingBal', 'ALL_SumCurrentOutstandingBalExcMtg'
]

for col in columns_for_median_imputation:
    median_value = merged_data[col].median()
    merged_data[col].fillna(median_value, inplace=True)

# For count and status fields, we'll use 0 or a special value
merged_data['ALL_CountActive'].fillna(0, inplace=True)
merged_data['ALL_CountDefaultAccounts'].fillna(0, inplace=True)
merged_data['ALL_WorstPaymentStatusActiveAccounts'].fillna(-1, inplace=True)  # -1 can indicate 'unknown' status

print("Missing values handled")
# Verify that all missing values have been addressed
missing_after = merged_data.isnull().sum()
print("\nMissing values after imputation:")
print(missing_after[missing_after > 0])

In [ ]:
# Convert ApplicationDate to datetime if not already
merged_data['ApplicationDate'] = pd.to_datetime(
    merged_data['ApplicationDate'], 
    format='%d/%m/%Y',  
    errors='coerce'     
)

# Create new features from ApplicationDate
merged_data['ApplicationYear'] = merged_data['ApplicationDate'].dt.year
merged_data['ApplicationMonth'] = merged_data['ApplicationDate'].dt.month
merged_data['ApplicationDayOfWeek'] = merged_data['ApplicationDate'].dt.dayofweek

print("DateTime features created")

In [ ]:
merged_data['ALL_TimeSinceMostRecentDefault'].replace(-1, np.nan, inplace=True)

In [ ]:
print("\nMissing values after handling special cases:")
print(merged_data.isnull().sum()[merged_data.isnull().sum() > 0])

In [ ]:
merged_data.describe().T

Feature engineering

In [ ]:
# Now extract features, handling potential NaT values
merged_data['ApplicationYear'] = merged_data['ApplicationDate'].dt.year
merged_data['ApplicationMonth'] = merged_data['ApplicationDate'].dt.month
merged_data['ApplicationDayOfWeek'] = merged_data['ApplicationDate'].dt.dayofweek

# Fill NaN values in the new columns with a suitable value, e.g., median
merged_data['ApplicationYear'] = merged_data['ApplicationYear'].fillna(merged_data['ApplicationYear'].median())
merged_data['ApplicationMonth'] = merged_data['ApplicationMonth'].fillna(merged_data['ApplicationMonth'].median())
merged_data['ApplicationDayOfWeek'] = merged_data['ApplicationDayOfWeek'].fillna(merged_data['ApplicationDayOfWeek'].median())

# Continue with the rest of your feature engineering
# Create a debt-to-income ratio 
if 'Income' in merged_data.columns:
    merged_data['DebtToIncomeRatio'] = merged_data['Amount'] / merged_data['Income']

# Create a feature for total number of accounts
merged_data['TotalAccounts'] = merged_data['ALL_Count']

# Create a feature for proportion of active accounts
merged_data['ProportionActiveAccounts'] = merged_data['ALL_CountActive'] / merged_data['ALL_Count']

# Create a feature for recent account activity
merged_data['RecentAccountActivity'] = merged_data['ALL_CountOpenedLast12Months'] - merged_data['ALL_CountClosedLast12Months']

# Log transform for skewed numeric features (example with 'Amount')
merged_data['LogAmount'] = np.log1p(merged_data['Amount'])


In [ ]:
merged_data.describe().T

In [ ]:
# Drop columns that are constant or entirely NaN
columns_to_drop = ['ALL_TimeSinceMostRecentDefault', 'ApplicationYear']
merged_data = merged_data.drop(columns=columns_to_drop)

print(f"Dropped columns: {columns_to_drop}")

In [ ]:
# Replace inf values with NaN
merged_data['ProportionActiveAccounts'] = merged_data['ProportionActiveAccounts'].replace([np.inf, -np.inf], np.nan)

# Fill NaN values with the median (or another meaningful value)
merged_data['ProportionActiveAccounts'].fillna(merged_data['ProportionActiveAccounts'].median(), inplace=True)

print("Handled inf values in ProportionActiveAccounts")

Correlation Analysis

In [ ]:
# Select numeric columns
numeric_columns = merged_data.select_dtypes(include=[np.number]).columns

# Compute the correlation matrix
correlation_matrix = merged_data[numeric_columns].corr()

# Plot the correlation heatmap
plt.figure(figsize=(20, 16))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap of Numeric Features')
plt.show()

# Print correlations with the target variable (assuming 'Success' is the target)
target_correlations = correlation_matrix['Success'].sort_values(ascending=False)
print("Top correlations with Success:")
print(target_correlations)

In [ ]:
# Select relevant features based on correlation analysis
selected_features = [
    'ALL_MeanAccountAge', 'ALL_AgeOfOldestAccount', 
    'ALL_SumCurrentOutstandingBal_Log', 'Amount',
    'ALL_WorstPaymentStatusActiveAccounts', 'Term',
    # Add other features based on your analysis
]
# Prepare data
X = merged_data[selected_features]
y = merged_data['Success']
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))